In [12]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
from urllib.request import Request, urlopen
import time
from selenium.common.exceptions import NoSuchElementException

# trustedreviews
def getReviews(list_of_page):
    
    reviews = []
    for page_url in list_of_page:
        page = requests.get(page_url)    
        if page.status_code==200:   # Check if success
            soup = BeautifulSoup(page.content, 'html.parser')
            raw_header = soup.find('h1', {'class': 'title-primary'})
            header = raw_header.get_text().strip()
            if 'Hands on:' not in header:
                raw_review = soup.find('div', {'class': 'static-content'}) 
                try:
                    review = raw_review.get_text().strip()
                    name = header.split(" Review")[0]
                    reviews.append({ 'Phone Name': name, 'Healine': header, 'Review': review, 
                                    'Source': 'trustedreviews', 'URL': page_url})
                except AttributeError:
                    pass # skip if no data
    df = pd.DataFrame(reviews)
    return df


def getListOfPhone(page_url):
    links = []
    page = requests.get(page_url) # request current page
    soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
    links = fetchLinks(soup, links)
    
    next_button = soup.find('a', {'class': 'button btn'})
    next_link = next_button.get('href')
    page = requests.get(next_link) # request current page
    soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
    links = fetchLinks(soup, links)
    
    count = 0
    while True :
        try:
            next_button = soup.find_all('a', {'class': 'button btn'})
            if len(next_button) > 1:
                next_button = next_button[1]
            else:
                break
            next_link =  next_button.get('href')
        except NoSuchElementException:
            break # stop after last page
        page = requests.get(next_link) # request current page
        soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
        links = fetchLinks(soup, links)
        count = count + 1
    
    return links

def fetchLinks(soup, links):
    anchors = soup.find_all('a', {'class': 'entry listing-entry'})
    for link in anchors:
        links.append(link.get('href'))
    return links

page_url = 'https://www.trustedreviews.com/reviews/mobile-phones'
list_of_page = getListOfPhone(page_url)

df = getReviews(list_of_page)
df.to_csv('CSVs/trustedreviews.csv')
df

,Phone Name,Healine,Review,Source,URL
0,Xiaomi Mi 11,Xiaomi Mi 11 Review,In this article…\n1. Verdict\n2. Key Specifica...,trustedreviews,https://www.trustedreviews.com/reviews/xiaomi-...
1,Google Pixel 6,Google Pixel 6 Review,In this article…\n1. Verdict\n2. Availability\...,trustedreviews,https://www.trustedreviews.com/reviews/google-...
2,Google Pixel 6 Pro,Google Pixel 6 Pro Review,In this article…\n1. Verdict\n2. Availability\...,trustedreviews,https://www.trustedreviews.com/reviews/google-...
3,Sony Xperia 5 III,Sony Xperia 5 III Review,In this article…\n1. Verdict\n2. Availability\...,trustedreviews,https://www.trustedreviews.com/reviews/sony-xp...
4,Coros Vertix 2,Coros Vertix 2 Review,In this article…\n1. Verdict\n2. Availability\...,trustedreviews,https://www.trustedreviews.com/reviews/coros-v...
...,...,...,...,...,...
180,Sony Xperia 1,Sony Xperia 1 Review,In this article…\n1. Verdict\n2. Pros and Cons...,trustedreviews,https://www.trustedreviews.com/reviews/sony-xp...
181,OnePlus 7,OnePlus 7 Review,In this article…\n1. Verdict\n2. Pros and Cons...,trustedreviews,https://www.trustedreviews.com/reviews/oneplus-7
182,"Amazon Fire HD 10 (Discontinued, 2018 model)","Amazon Fire HD 10 (Discontinued, 2018 model) R...",In this article…\n1. Verdict\n2. Pros and Cons...,trustedreviews,https://www.trustedreviews.com/reviews/amazon-...
183,Asus ZenFone 6,Asus ZenFone 6 Review,In this article…\n1. Verdict\n2. Pros and Cons...,trustedreviews,https://www.trustedreviews.com/reviews/asus-ze...
